In [1]:
import csv
import ast
import json
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
from collections import Counter
from itertools import chain
import ast
import re

In [2]:
# In order to compare the 2 models we need to perform the NER of the same speeches in the distilBERT model

ner_model = pipeline("ner", 
                     model="elastic/distilbert-base-cased-finetuned-conll03-english",  # DistilBERT pre-trained on CoNLL-03 NER
                     aggregation_strategy="simple")

: 

In [3]:


# Function to preprocess and merge entities
def preprocess_entities(entity_list):
    merged_entities = []
    temp_entity = []
    for entity, label in entity_list:
        # Check if the entity is a continuation of the previous one (look for subword markers or periods)
        if re.match(r'^[##\.]', entity) or (temp_entity and re.match(r'[\.]', temp_entity[-1][0])):
            temp_entity.append((entity, label))
        else:
            if temp_entity:
                merged_entities.append(("".join(e[0].replace('##', '').replace('.', '') for e in temp_entity), temp_entity[0][1]))
            temp_entity = [(entity, label)]
    
    # Append the last accumulated entity
    if temp_entity:
        merged_entities.append(("".join(e[0].replace('##', '').replace('.', '') for e in temp_entity), temp_entity[0][1]))

    return merged_entities


def extract_entities(texts):
    entities_list = []
    for text in tqdm(texts, desc="Processing NER", ncols=100):
        entities = ner_model(text)
        entities_list.append([(ent['word'], ent['entity_group']) for ent in entities])
    
    merged_entities = [preprocess_entities(entities) for entities in entities_list]
    json_style_entities = merged_entities.apply(lambda x: json.dumps([{"text": ent[0], "label": ent[1]} for ent in x]))
    return json_style_entities


# Process and save entities
def format_and_save(df, filename):
    df['entities'] = df['entities'].apply(preprocess_entities)
    df['entities'] = df['entities'].apply(lambda x: json.dumps([{"text": ent[0], "label": ent[1]} for ent in x]))
    df.to_csv(filename, index=False)




: 

In [ ]:
# Load the data
df_obama = pd.read_csv('../data/preprocessed/sampled_cleaned_obama.csv')
df_trump = pd.read_csv('../data/preprocessed/sampled_cleaned_trump.csv')
df_biden = pd.read_csv('../data/preprocessed/sampled_cleaned_biden.csv')


# Extract entities
df_obama['entities'] = extract_entities(df_obama['processed_text'])
df_trump['entities'] = extract_entities(df_trump['processed_text'])
df_biden['entities'] = extract_entities(df_biden['processed_text'])

# Save the entities
df_obama.to_csv('../data/entities/distilBERT_NER_results_obama.csv', index=False)
df_trump.to_csv('../data/entities/distilBERT_NER_results_trump.csv', index=False)
df_biden.to_csv('../data/entities/distilBERT_NER_results_biden.csv', index=False)



In [8]:
def read_manual_labels(file_path):
    manual_labels = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            # Safely evaluate the line as a tuple
            try:
                entity = ast.literal_eval(line.strip())
                manual_labels.append(entity)  # Append the (text, label) tuple
            except (ValueError, SyntaxError) as e:
                print(f"Error parsing entity: {line}, Error: {e}")

    return manual_labels

# Example usage (for Obama labels):
manual_labels = read_manual_labels("../data/preprocessed/manual_labeling_obama.csv")
print(manual_labels)

[('United Nations', 'ORG'), ('Contact Group on Piracy off the Coast of Somalia', 'ORG'), ('United States', 'ORG'), ('New York Declaration', 'ORG'), ('International Ship and Port Facility Security Code', 'ORG'), ('Panama', 'ORG'), ('Bahamas', 'ORG'), ('Liberia', 'ORG'), ('Marshall Islands', 'ORG'), ('NATO', 'ORG'), ('European Union', 'ORG'), ('Somalia', 'LOC'), ('Coast of Somalia', 'LOC'), ('United States', 'LOC'), ('New York', 'LOC'), ('Horn of Africa', 'LOC'), ('piracy', 'MISC'), ('Syrian Embassy', 'ORG'), ('United States', 'ORG'), ('State Department', 'ORG'), ('Honorary Consuls', 'ORG'), ('Vienna Convention on Diplomatic Relations', 'ORG'), ('United States', 'LOC'), ('Michigan', 'LOC'), ('Texas', 'LOC'), ('Washington', 'LOC'), ('Syria', 'LOC'), ('Syrian ambassador', 'PER'), ('Clinton', 'PER'), ('Lavrov', 'PER'), ('Obama', 'PER'), ('Medvedev', 'PER'), ('United States', 'ORG'), ('G-20', 'ORG'), ('NATO', 'ORG'), ('NATO Russia Council', 'ORG'), ('Russia', 'ORG'), ('START', 'ORG'), ('Russ

In [9]:
def read_ner_results(file_path, max_lines=None):
    ner_results = []
    with open(file_path, "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader)  # Skip the header line 'publish_date,entities'

        for i, row in enumerate(reader):
            if max_lines is not None and i >= max_lines:
                break  # Stop if we reach the specified line limit
            if len(row) < 2:
                continue  # Skip if the row is malformed or incomplete

            entities_str = row[1]  # We're only interested in the second column, which contains the entities
            try:
                # Safely evaluate the string containing the list of entities
                entities = ast.literal_eval(entities_str)
                ner_results.extend([(entity['text'], entity['label']) for entity in entities])  # Add (text, label) tuples
            except (ValueError, SyntaxError) as e:
                print(f"Error parsing entity: {entities_str}, Error: {e}")

    return ner_results

In [10]:
# Evaluation function remains the same
def evaluate_ner(manual_labels, ner_results):
    manual_set = set(manual_labels)
    ner_set = set(ner_results)

    true_positives = manual_set & ner_set
    false_positives = ner_set - manual_set
    false_negatives = manual_set - ner_set

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if len(ner_set) > 0 else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if len(manual_set) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

    return precision, recall, f1

In [ ]:
#create the NER results using the distilBERT model



In [11]:
# Paths to the files (adjusted to the naming scheme in the screenshot)
file_pairs = [
    ("../flairData/manual_labeling_biden.csv", "../flairData/ner_results_biden.csv"),
    ("../flairData/manual_labeling_obama.csv", "../flairData/ner_results_obama.csv"),
    ("../flairData/manual_labeling_trump.csv", "../flairData/ner_results_trump.csv")
]

# Evaluate and print the results for each file pair
for manual_file, ner_file in file_pairs:
    president = manual_file.split('_')[-1].split('.')[0].capitalize()  # Extracting 'Biden', 'Obama', 'Trump'
    
    # Reading manual labels and NER results using the existing functions
    manual_labels = read_manual_labels(manual_file)
    ner_results = read_ner_results(ner_file, 10)

    # Evaluate the performance
    precision, recall, f1 = evaluate_ner(manual_labels, ner_results)

    # Print the evaluation results in a nice format
    print(f"Evaluating {president} NER Results:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print("-" * 40)

Evaluating Biden NER Results:
Precision: 0.5618
Recall:    0.6993
F1 Score:  0.6231
----------------------------------------
Evaluating Obama NER Results:
Precision: 0.3412
Recall:    0.5088
F1 Score:  0.4085
----------------------------------------
Evaluating Trump NER Results:
Precision: 0.5733
Recall:    0.6615
F1 Score:  0.6143
----------------------------------------
